# patient-online-deprivation-score-model
#### Developed by: Mary Amanuel
#### Contact: mary.amanuel@nhsx.nhs.uk
#### Created: 22nd November 2021
#### Last updated: 22nd November 2021


In [ ]:
import pandas as pd
import os
import plotly
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

##### 'Public Health England. Public Health Profiles. [22/11/2021] https://fingertips.phe.org.uk © Crown copyright [2021]'
##### Technical documentation for data here: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833951/IoD2019_Technical_Report.pdf

##### Research documentation for data here: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833947/IoD2019_Research_Report.pdf

In [ ]:
# Import fingertips_py package - This is a python package to interact with Public Health England’s [Fingertips](https://fingertips.phe.org.uk/) data tool. Fingertips is a major repository of population and public health indicators for England.
## https://fingertips-py.readthedocs.io/en/latest/#indices-and-table
## Installing the fingertips_py library (if not already)

!pip install fingertips_py
import fingertips_py

In [ ]:
national_gp_profiles = fingertips_py.get_profile_by_name('National General Practice Profiles')
national_gp_profiles

In [ ]:
## Check the relevant geographies that can be called upon as area id. For instance, it can be seen that 'General Practice' is 7
fingertips_py.metadata.get_area_types_as_dict()

In [ ]:
# Obtain dataframe with metadata on metrics under chosen topic (can instead just navigate website direct)
national_gp_profiles_meta = fingertips_py.get_metadata_for_profile_as_dataframe(national_gp_profiles['Id'])
national_gp_profiles_meta

In [ ]:
gp_and_england_deprivation_score = fingertips_py.retrieve_data.get_data_by_indicator_ids(indicator_ids=[93553],area_type_id =[7] ) # 7 for practice (we know this topic area has practice breakdown for sure, by design), 93553 for our metric of interest
gp_and_england_deprivation_score

In [ ]:
gp_and_england_deprivation_score.info()

In [ ]:
gp_and_england_deprivation_score = gp_and_england_deprivation_score[['Area Code', 'Area Name', 'Area Type', 'Time period', 'Value']]
gp_and_england_deprivation_score

In [ ]:
gp_deprivation_score_2019 = gp_and_england_deprivation_score[gp_and_england_deprivation_score['Area Type'].isin(['GP'])]
gp_deprivation_score_2019 = gp_deprivation_score_2019.rename(columns={'Area Code' :'Area_Code', 
                                          'Area Name' : 'GP_Name',
                                          'Area Type' : 'Area_Type',
                                          'Time period' : 'Time_Period', 
                                          'Value' : 'Deprivation_Score_IMD_2019'})
gp_deprivation_score_2019['GP_Name'] = gp_deprivation_score_2019['GP_Name'].str.upper()

In [ ]:
gp_deprivation_score_2019

In [ ]:
gp_deprivation_score_2019.describe()

In [ ]:
gp_deprivation_score_2019['GP_Rank'] = gp_deprivation_score_2019['Deprivation_Score_IMD_2019'].rank(ascending = False)
gp_deprivation_score_2019

In [ ]:
gp_deprivation_score_2019_ranking = gp_deprivation_score_2019.sort_values(by='GP_Rank', ascending=True)
gp_deprivation_score_2019_ranking['Percentile_Rank'] = gp_deprivation_score_2019.Deprivation_Score_IMD_2019.rank(pct = True)
gp_deprivation_score_2019_ranking

In [ ]:
gp_deprivation_score_2019_ranking['decile'] = pd.qcut(gp_deprivation_score_2019_ranking['Deprivation_Score_IMD_2019'], 10, labels=['10% least deprived', '20% least deprived', '30% least deprived', '40% least deprived', '50% least deprived', '50% most deprived', '40% most deprived', '30% most deprived', '20% most deprived', '10% most deprived'])
gp_deprivation_score_2019_ranking

Patient Online Data 2019